In [ ]:
## setup analysis
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.purification import purify_pq as ppq; reload(ppq)
from analysis.lib.purification import purify_analysis as pa; reload(pa)
%matplotlib inline
import csv
import os

thetas = ['pi/4','pi/5','pi/6','pi/8']

save = True
fileDir = r'C:\Users\tud276629\Documents'

In [ ]:
def sweep_parameter(param_name,sweep_pts,apply_ROC, plot_all = False):
    
    ## get correlation probabilties
    psi_m_XX,psi_p_XX,psi_m_XX_corrs,psi_p_XX_corrs = Purify_XX.sweep_filter_parameter_vs_correlations(param_name,sweep_pts,
                                                                         apply_ROC = apply_ROC,do_plot= False)
    psi_m_YY,psi_p_YY,psi_m_YY_corrs,psi_p_YY_corrs = Purify_YY.sweep_filter_parameter_vs_correlations(param_name,sweep_pts,
                                                                         apply_ROC = apply_ROC,do_plot= False)
    psi_m_ZZ,psi_p_ZZ,psi_m_ZZ_corrs,psi_p_ZZ_corrs = Purify_ZZ.sweep_filter_parameter_vs_correlations(param_name,sweep_pts,
                                                                         apply_ROC = apply_ROC,do_plot= False)       
    ## calculate fidelities
    psi_m_F = (psi_m_XX[1] + psi_m_YY[1] + psi_m_ZZ[1]+1)/4.
    psi_p_F = (psi_p_XX[1] + psi_p_YY[1] + psi_p_ZZ[1]+1)/4.
      
    ## calciulate error bars
    psi_m_F_u = np.sqrt(psi_m_XX[2]**2 + psi_m_YY[2]**2 + psi_m_ZZ[2]**2)/4.
    psi_p_F_u = np.sqrt(psi_p_XX[2]**2 + psi_p_YY[2]**2 + psi_p_ZZ[2]**2)/4.
    
    return zip(sweep_pts,\
               psi_m_XX[1],psi_m_XX[2],psi_m_YY[1],psi_m_YY[2],psi_m_ZZ[1],psi_m_ZZ[2],\
               psi_p_XX[1],psi_p_XX[2],psi_p_YY[1],psi_p_YY[2],psi_p_ZZ[1],psi_p_ZZ[2],\
               psi_m_F,psi_m_F_u,psi_p_F,psi_p_F_u)

In [ ]:
for theta in thetas:

    folder_lt3,folder_lt4 = pa.get_folders_from_theta(theta)
    ssro_calib_lt3,ssro_calib_lt4 = pa.get_ssro_calibs()

    Purify_XX = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)
    Purify_YY = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)
    Purify_ZZ = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)

    Purify_XX.get_tstamps_and_offsets(contains = 'XX')
    Purify_YY.get_tstamps_and_offsets(contains = 'YY')
    Purify_ZZ.get_tstamps_and_offsets(contains = 'ZZ')

        ### load the data
    Purify_XX.load_raw_data()
    Purify_XX.correct_pq_times()
    Purify_YY.load_raw_data()
    Purify_YY.correct_pq_times()
    Purify_ZZ.load_raw_data()
    Purify_ZZ.correct_pq_times()
    bins = np.arange(50,450,50)
    
    corrVals = sweep_parameter('bin_w2',bins,apply_ROC = True,plot_all = True)
      
    if save:
        f = open(os.path.join(fileDir,'correlations_for_'+str.replace(theta,"/",'') +'.csv'), 'w')
        writer = csv.writer(f, delimiter='\t')
        writer.writerows([['bin','m_XX','m_XX_u','m_YY','m_YY_u','m_ZZ','m_ZZ_u',\
                         'p_XX','p_XX_u','p_YY','p_YY_u','p_ZZ','p_ZZ_u',\
                         'm_F','m_F_u','p_F','p_F_u']])
        writer.writerows(corrVals)
        f.close()
        
    print 'Theta, ', theta
    print 'Stats for first 50 bins:'
    print corrVals[0]